In [12]:
import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import tensorflow
import keras
import numpy as np


plc_buffer_name = 'Main.profiler_buffer'
plc_numlayers_name = 'Main.FB_NN.nn.num_layers'
plc_numneurons_name = 'Main.FB_NN.nn.layers[[index]].num_neurons'
plc_MeasureFinished_name = 'Main.MeasureFinished'
plc_output_name = 'Main.out'
plc_input_name = 'Main.input'

num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]

In [37]:
python_model = keras.saving.load_model('test_model.keras')

np.random.seed(2024)
measurement = []
rand_input = np.random.rand(100,5)*10e5
memory = 0
for i in range(rand_input.shape[0]):
    input = rand_input[i,:]
    python_input = input.reshape(1,5)
    python_output = python_model.predict(python_input, verbose=0)
    with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
        plc.write_by_name(plc_input_name, input.tolist() ,pyads.PLCTYPE_LREAL*5)
    while(1):
        with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
            rtnn_output = plc.read_by_name(plc_output_name, pyads.PLCTYPE_LREAL)
        if memory!=rtnn_output:
            memory = rtnn_output
            break
    dif = python_output[0,0]-rtnn_output
    measurement.append([python_output[0,0],rtnn_output, dif, (dif/python_output[0,0])*100])



In [38]:
df_abv = pd.DataFrame(measurement, columns=['python_output', 'rtnn_output', 'difference', 'percent %'])
print(f'max = {np.abs(df_abv['percent %']).max()}')
print(df_abv['percent %'])

max = 5.372926561137872e-05
0     2.961196e-06
1     2.237100e-05
2    -7.971059e-06
3    -1.094293e-05
4    -9.082191e-06
          ...     
95    7.745032e-07
96    3.638224e-05
97    4.842113e-06
98   -1.109029e-05
99   -1.209942e-05
Name: percent %, Length: 100, dtype: float64
